In [1]:
import pandas as pd
ess = pd.read_csv('ess.csv')

C:\Users\timry\AppData\Local\Temp\ipykernel_45076\1681056850.py:2: DtypeWarning: Columns (164) have mixed types. Specify dtype option on import or set low_memory=False.
  ess = pd.read_csv('ess.csv')


In [2]:
print(ess.shape)

(44387, 534)


In [3]:
print(ess.loc[:,"happy"].head())

0    5
1    5
2    8
3    8
4    5
Name: happy, dtype: int64


In [4]:
print(ess.loc[:,"sclmeet"].head())

0    6
1    4
2    4
3    4
4    6
Name: sclmeet, dtype: int64


In [5]:
ess = ess.loc[ess["sclmeet"] <=10,:].copy()
ess = ess.loc[ess["rlgdgr"] <=10,:].copy()
ess = ess.loc[ess["hhmmb"] <=50,:].copy()
ess = ess.loc[ess["netusoft"] <=5,:].copy()
ess = ess.loc[ess["agea"] <=200,:].copy()
ess = ess.loc[ess["health"] <=5,:].copy()
ess = ess.loc[ess["happy"] <=10,:].copy()
ess = ess.loc[ess["eduyrs"] <=100,:].copy().reset_index(drop=True)

In [6]:
import numpy as np
social=list(ess.loc[:,'sclmeet'])
happy=list(ess.loc[:,'happy'])
low_social_happiness = [hap for soc,hap in zip(social,happy) if soc <=5]
high_social_happiness = [hap for soc,hap in zip(social,happy) if soc > 5]

meanlower = np.mean(low_social_happiness)
meanhigher = np.mean(high_social_happiness)

In [7]:
meanlower

7.194794027984054

In [8]:
meanhigher

7.826184974356084

In [9]:
lowerrors = [abs(lowhappy-meanlower) for lowhappy in low_social_happiness]
higherrors = [abs(highhappy-meanhigher) for highhappy in high_social_happiness]
total_error = sum(lowerrors) + sum(higherrors)

## Get Split Point

In [15]:
def get_splitpoint(allvalues, predictedvalues):
    lowest_error = float("inf")
    best_split = None
    best_lowermean = np.mean(predictedvalues)
    best_highermean = np.mean(predictedvalues)
    for pctl in range(0,100):
        split_candidate = np.percentile(allvalues, pctl)
        loweroutcomes = [outcome for value, outcome in zip(allvalues, predictedvalues) if \
                        value <= split_candidate]
        higheroutcomes = [outcome for value, outcome in zip(allvalues, predictedvalues) if \
                        value > split_candidate]
        if np.min([len(loweroutcomes),len(higheroutcomes)]) > 0:
            meanlower = np.mean(loweroutcomes)
            meanhigher = np.mean(higheroutcomes)
            lowererrors = [abs(outcome-meanlower) for outcome in loweroutcomes]
            highererrors = [abs(outcome-meanhigher) for outcome in higheroutcomes]
            total_error = sum(lowererrors) + sum(highererrors)
            if total_error < lowest_error:
                best_split = split_candidate
                lowest_error = total_error
                best_lowermean = meanlower
                best_highermean = meanhigher
    return(best_split, lowest_error, best_lowermean, best_highermean)

In [16]:
allvalues = list(ess.loc[:,"hhmmb"])
predictedvalues = list(ess.loc[:,"happy"])
print(get_splitpoint(allvalues,predictedvalues))

(1.0, 60860.029867951016, 6.839403436723225, 7.620055170794695)


## Choosing the splitting variable

In [17]:
def getsplit(data, variables, outcome_variable):
    best_var = ''
    lowest_error = float('inf')
    best_split = None
    predictedvalues = list(data.loc[:,outcome_variable])
    best_lowerman = -1
    best_higherman = -1
    for var in variables:
        allvalues = list(data.loc[:,var])
        splitted = get_splitpoint(allvalues, predictedvalues)
        if(splitted[1] < lowest_error):
            best_split = splitted[0]
            lowest_error = splitted[1]
            best_var = var
            best_lowermean = splitted[2]
            best_highermean = splitted[3]
    generated_tree = [[best_var, float('-inf'), best_split, best_lowermean],\
                      [best_var, best_split, float('inf'), best_highermean]]
    return(generated_tree)

In [18]:
variables = ["rlgdgr","hhmmb","netusoft","agea","eduyrs"]
outcome_variable = "happy"
print(getsplit(ess,variables,outcome_variable))

[['netusoft', -inf, 4.0, 7.041597337770383], ['netusoft', 4.0, inf, 7.73042471042471]]


## Adding depth

In [23]:
maxdepth=3

def getsplit(depth, data, variables, outcome_variable):
    best_var = ''
    lowest_error = float('inf')
    best_split = None
    predictedvalues = list(data.loc[:,outcome_variable])
    best_lowerman = -1
    best_higherman = -1
    for var in variables:
        allvalues = list(data.loc[:,var])
        splitted = get_splitpoint(allvalues, predictedvalues)
        if(splitted[1] < lowest_error):
            best_split = splitted[0]
            lowest_error = splitted[1]
            best_var = var
            best_lowermean = splitted[2]
            best_highermean = splitted[3]
    generated_tree = [[best_var, float('-inf'), best_split, best_lowermean],\
                      [best_var, best_split, float('inf'), best_highermean]]
    
    if depth<maxdepth:
        splitdata1=data.loc[data[best_var] <= best_split,:]
        splitdata2=data.loc[data[best_var] > best_split,:]
        if len(splitdata1.index)>10 and len(splitdata2.index)>10:
            generated_tree[0][3] = getsplit(depth+1, splitdata1, variables, outcome_variable)
            generated_tree[1][3] = getsplit(depth+1, splitdata2, variables, outcome_variable)
        else:    
            depth = maxdepth+1
            generated_tree[0][3] = best_lowermean
            generated_tree[1][3] = best_highermean
    else:
        generated_tree[0][3] = best_lowermean
        generated_tree[1][3] = best_highermean
    
    return(generated_tree)

In [24]:
variables = ["rlgdgr","hhmmb","netusoft","agea","eduyrs"]
outcome_variable = "happy"
max_depth = 2
print(getsplit(0,ess,variables,outcome_variable))

[['netusoft', -inf, 4.0, [['hhmmb', -inf, 4.0, [['agea', -inf, 15.0, [['eduyrs', -inf, 7.08, 7.0], ['eduyrs', 7.08, inf, 8.115384615384615]]], ['agea', 15.0, inf, [['agea', -inf, 16.0, 8.523809523809524], ['agea', 16.0, inf, 6.995696015740285]]]]], ['hhmmb', 4.0, inf, [['eduyrs', -inf, 11.0, [['rlgdgr', -inf, 8.0, 7.0275], ['rlgdgr', 8.0, inf, 8.058823529411764]]], ['eduyrs', 11.0, inf, [['rlgdgr', -inf, 9.0, 7.7894736842105265], ['rlgdgr', 9.0, inf, 9.009708737864077]]]]]]], ['netusoft', 4.0, inf, [['hhmmb', -inf, 1.0, [['agea', -inf, 66.0, [['eduyrs', -inf, 16.0, 7.037735849056604], ['eduyrs', 16.0, inf, 7.3924949290060855]]], ['agea', 66.0, inf, [['rlgdgr', -inf, 6.0, 7.458333333333333], ['rlgdgr', 6.0, inf, 7.9393939393939394]]]]], ['hhmmb', 1.0, inf, [['rlgdgr', -inf, 5.0, [['eduyrs', -inf, 16.0, 7.689139633286318], ['eduyrs', 16.0, inf, 7.9211567732115675]]], ['rlgdgr', 5.0, inf, [['eduyrs', -inf, 8.0, 7.759103641456583], ['eduyrs', 8.0, inf, 7.998256905336551]]]]]]]]


## Evaluating decision tree

In [25]:
def get_prediction(observation, tree):
    j=0
    keepgoing = True
    prediction= -1
    while(keepgoing):
        j = j+1
        variable_tocheck = tree[0][0]
        bound1 = tree[0][1]
        bound2 = tree[0][2]
        bound3 = tree[1][2]
        if observation.loc[variable_tocheck]<bound1:
            tree = tree[0][3]
        else:
            tree = tree[1][3]
        if isinstance(tree,float):
            keepgoing=False
            prediction=tree
    return(prediction)

In [26]:
predictions=[]
outcome_variable="happy"
maxdepth=4
thetree=getsplit(0,ess,variables,outcome_variable)
for k in range(0,30):
    observation=ess.loc[k,:]
    predictions.append(get_prediction(observation,thetree))
print(predictions)

[7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146, 7.999194457870146]


In [28]:
predictions=[]

for k in range(0,len(ess.index)):
    observation=ess.loc[k,:]
    predictions.append(get_prediction(observation,thetree))
ess.loc[:,'predicted'] = predictions    
errors = abs(ess.loc[:,'predicted'] - ess.loc[:,'happy'])

print(np.mean(errors))

1.34966377256382
